In [ ]:
from sklearn.metrics import classification_report, roc_auc_score
model.eval()
y_true, y_pred_proba = [], []
with torch.no_grad():
    for xb,yb in val_loader:
        xb = xb.to(device)
        logits = model(xb)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        y_pred_proba.append(probs)
        y_true.append(yb.numpy())
import numpy as np
y_true = np.concatenate(y_true)
y_pred_proba = np.concatenate(y_pred_proba)
y_pred = y_pred_proba.argmax(axis=1)
print(classification_report(y_true, y_pred))
if y_pred_proba.shape[1]>1:
    try:
        auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')
        print("Multi-class ROC AUC:", auc)
    except Exception as e:
        print("AUC error:", e)


In [ ]:
model.eval()
example = torch.rand(1,3,224,224).to(device)
traced = torch.jit.trace(model, example)
traced.save("models/resnet_traced.pt")
print("Saved TorchScript model at models/resnet_traced.pt")


In [ ]:
streamlit_app = """
import streamlit as st
from PIL import Image
import torch, torchvision
st.title('CliniScan - Tiny Demo')
uploaded = st.file_uploader("Upload CXR PNG", type=['png','jpg','jpeg'])
if uploaded:
    img = Image.open(uploaded).convert('RGB')
    st.image(img, caption='Uploaded image', use_column_width=True)
    if st.button("Run inference"):
        model = torchvision.models.resnet18(pretrained=False)
        model.fc = torch.nn.Linear(model.fc.in_features, 2)
        model.load_state_dict(torch.load('models/resnet18_baseline.pth', map_location='cpu'))
        model.eval()
        transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224)),
                                                    torchvision.transforms.ToTensor(),
                                                    torchvision.transforms.Normalize([0.485,0.456,0.406],
                                                                                     [0.229,0.224,0.225])])
        x = transform(img).unsqueeze(0)
        with st.spinner("Running..."):
            probs = torch.softmax(model(x), dim=1).detach().numpy()[0]
        st.write("Predicted class probs:", probs.tolist())
"""
with open("app.py","w") as fh:
    fh.write(streamlit_app)
print("Saved Streamlit demo as app.py. Run: streamlit run app.py")
